#1. Import Dependencies

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math
from google.colab import files

#2. Function to scrape required data from url

In [ ]:
def webScraping(url):

  Ratings = []
  Summary = []
  Date_JobTitle = []
  Date = []
  JobTitle = []
  Pros = []
  Cons = []

  response = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0'})
  soup = BeautifulSoup(response.text, 'html.parser')

  for x in soup.find_all('ol', class_ = ' empReviews emp-reviews-feed pl-0'):
  
    #employee rating
    for i in x.find_all('span', class_ = 'ratingNumber mr-xsm'):
      Ratings.append(i.text)

    #summary
    for i in x.find_all('h2', class_ = 'mb-xxsm mt-0 css-93svrw el6ke055'):
      Summary.append(i.text)

    #date_jobtitle
    for i in x.find_all('span', class_ = 'middle common__EiReviewDetailsStyle__newGrey'):
      Date_JobTitle.append(i.text)

    #date
    for i in Date_JobTitle:
      Date.append(i.split(' -')[0])
    
    #job title
    for i in Date_JobTitle:
      JobTitle.append(i.split('- ')[1])

    #pros
    for i in x.find_all('span', {'data-test' : 'pros'}):
      Pros.append(i.text)

    #cons
    for i in x.find_all('span', {'data-test' : 'cons'}):
      Cons.append(i.text)
    
  #create pandas dateframe and store review details
  Reviews = pd.DataFrame(list(zip(Summary, Ratings, Date, JobTitle, Pros, Cons)), columns = ['Summary', 'Ratings', 'Date', 'JobTitle', 'Pros', 'Cons'])

  return Reviews

In [ ]:
response = requests.get('https://www.glassdoor.com/Reviews/Amazon-Reviews-E6036.htm', headers = {'User-Agent': 'Mozilla/5.0'})
response

In [ ]:
# get total pages based on total number of reviews
soup = BeautifulSoup(response.text, 'html.parser')
totalReviews = soup.find('div', class_ = 'paginationFooter').text
totalReviews = int((totalReviews.split(' Reviews')[0].split('of ')[1]).replace(',',''))

totalPages = math.ceil(totalReviews / 10)   #10 reviews per page

In [ ]:
FinalReviews = pd.DataFrame()
url1 = 'https://www.glassdoor.com/Reviews/Amazon-Reviews-E6036_P1.htm?sort.sortType=RD&sort.ascending=false'
FinalReviews = webScraping(url1)
for i in range(2, 5):
#for i in range(2, totalPages + 1):
  input_url = f'https://www.glassdoor.com/Reviews/Amazon-Reviews-E6036_P{i}.htm?sort.sortType=RD&sort.ascending=false'
  FinalReviews = FinalReviews.append(webScraping(input_url), ignore_index=True)

In [ ]:
display(FinalReviews)

In [ ]:
# conver to csv file
FinalReviews.to_csv('GlassdoorReviews.csv', index = False)